In [8]:
from openai import OpenAI
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text

In [4]:
api_key = os.environ.get('api_key')


In [5]:
df = pd.read_csv('sales_data_sample.csv')

In [11]:
# create a temp databese in memory
temp_db = create_engine('sqlite:///:memory:', echo=True)


In [ ]:
#push the pandas df ---> to temp db for us to be able to interact
data = df.to_sql(name='Sales', con=temp_db)

In [20]:
#SQL query on TEMP DB
with temp_db.connect() as conn:
    result = conn.execute(text(
        "SELECT SUM(SALES) FROM Sales GROUP BY QTR_ID LIMIT 5"
        ))

2024-01-24 11:55:10,882 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-24 11:55:10,883 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales GROUP BY QTR_ID LIMIT 5
2024-01-24 11:55:10,884 INFO sqlalchemy.engine.Engine [cached since 122.1s ago] ()
2024-01-24 11:55:10,885 INFO sqlalchemy.engine.Engine ROLLBACK


In [19]:
result.all()

[(2350817.7300000004,),
 (2048120.2999999986,),
 (1758910.8099999994,),
 (3874780.01,)]